<a href="https://colab.research.google.com/github/CosmicSelection/DuelSim/blob/main/DuelSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [53]:
!pip3 install yugioh
!pip3 install numpy


In [54]:
import numpy as np
import random as rand
import re
import yugioh


p1DeckBuild = np.array(["Mystical Elf", "Winged Dragon, Guardian of the Fortress #1", "Summoned Skull", "Beaver Warrior", "Dark Magician",
                        "Gaia The Fierce Knight", "Curse of Dragon", "Celtic Guardian", "Mammoth Graveyard", "Silver Fang",
                        "Giant Soldier of Stone", "Dragon Zombie", "Doma The Angel of Silence", "Ansatsu", "Witty Phantom", "Claw Reacher",
                        "Sword of Dark Destruction", "Book of Secret Arts", "Dark Hole", "Dian Keto the Cure Master", "Ancient Elf",
                        "Magical Ghost", "Fissure", "Trap Hole", "De-Spell", "Monster Reborn", "Reinforcements", "Change of Heart",
                        "Wall of Illusion", "Neo the Magic Swordsman", "Baron of the Fiend Sword", "Sorcerer of the Doomed", 	"Waboku", "Soul Exchange",
                        "Dragon Capture Jar", "Sakuretsu Armor", "Remove Trap", "Castle Walls", "Ultimate Offering", "Lightning Vortex"])

p2DeckBuild = np.array(["Blue-Eyes White Dragon", "Hitotsu-Me Giant", 	"Battle Ox", 	"Koumori Dragon", "Judge Man",
                        "Rogue Doll", "Kojikocy", "Uraby", "Gyakutenno Megami", "Mystic Horseman", "Terra the Terrible", "Dark Titan of Terror",
                        "Master & Expert", 	"Mystic Clown", "Ogre of the Black Shadow", "Dark Energy", "Invigoration", "Dark Hole",
                        "Ookazi", "Ryu-Kishin Powered", "Swordstalker", "La Jinn the Mystical Genie of the Lamp", "Rude Kaiser", "Destroyer Golem",
                        "Skull Red Bird", "D. Human", "Pale Beast", "Fissure", "Trap Hole", "Mystical Space Typhoon", 	"De-Spell", "Castle Walls",
                        "Monster Reborn", "Reinforcements", "Lightning Vortex", "Ultimate Offering", "Galaxy Serpent",  "Sogen",
                        "Sakuretsu Armor", "Remove Trap"])


p1Deck = np.array([])
p2Deck = np.array([])
spellTrapList = []
for name in p1DeckBuild:
    card = yugioh.get_card(card_name=name)
    p1Deck = np.append(p1Deck, [card])

for name in p2DeckBuild:
    card = yugioh.get_card(card_name=name)
    p2Deck = np.append(p2Deck, [card])


In [55]:
# Main
def letsDuel(p1Deck, p2Deck):
    # Field
    # p1 spell/trap zone -> [None, None, None, None, None]  Field[0][0]
    # p1 monster zone -> [None, None, None, None, None]     Field[0][1]
    # p2 monster zone -> [None, None, None, None, None]     Field[1][0]
    # p2 spell/trap zone -> [None, None, None, None, None]  Field[1][1]
    Field = np.array([[[], []], [[], []]])

    # true = face down (set), false = face up (not set)
    setbools = np.array([[[], []], [[], []]])

    # true = atk position, false = def position
    posBools = np.array([[], []])

    # grave for all used or destroyed cards
    p1Grave = [[], []]
    p2Grave = [[], []]

    # player
    p1 = 1
    p2 = 2

    # shuffles  deck
    p1Deck = rand.choices(p1Deck, k=len(p1Deck))
    p2Deck = rand.choices(p2Deck, k=len(p2Deck))
    p1Hand = p1Deck[0:5]
    p2Hand = p2Deck[0:5]

    # life points
    p1LP = 4000
    p2LP = 4000
    p1Draw = 5
    p2Draw = 5
    turn = 1
    coinToss = None

    while (p1LP > 0) and (p2LP > 0) and (p1Draw < len(p1Deck)) and (p2Draw < len(p2Deck)):
        if turn == 1:
            coinToss = rand.randrange(2)
        elif coinToss == 0 and turn % 2 == 0:
            np.append(p2Hand, p2Deck[p2Draw])
            while len(p2Hand) > 7:
                # discard(p2Hand)
                pass
            # standby()
        elif coinToss == 0 and turn % 2 != 0:
            np.append(p1Hand, p1Deck[p1Draw])
            while len(p1Hand) > 7:
                # discard(p2Hand)
                pass
            # standby()
        elif coinToss == 1 and turn % 2 != 0:
            np.append(p2Hand, p2Deck[p2Draw])
            while len(p2Hand) > 7:
                # discard(p2Hand)
                pass
            # standby()
        elif coinToss == 1 and turn % 2 == 0:
            np.append(p1Hand, p1Deck[p1Draw])
            while len(p1Hand) > 7:
                # discard(p2Hand)
                pass
            # standby()
        turn = turn + 1
    if (p1LP == 0 and p2LP == 0) or (p1Draw == len(p1Deck) and p2Draw == len(p2Deck)):
        print("TIED GAME")
    elif p2LP == 0 or p2Draw == len(p2Deck):
        print("Player 1 Wins!")
    elif p1LP == 0 or p1Draw == len(p1Deck):
        print("Player 2 Wins!")


In [56]:
# All Game Mechanics


# Summoning Mechanics
def normalSummon(player, pHand, Field, setBools, posBools, monster, GY):
    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0
    if len(Field[x][y]) > 5:
        min = monster.attack
        minMon = monster
        for mon in Field[x][y]:
            if mon.attack < min:
                min = mon.attack
                minMon = mon
        if monster == minMon:
            return None
        np.delete(Field[0][1], minMon)
        np.append(GY, minMon)
    controlledMonsters = len(Field[x][y])
    setBools[1][0] = np.append(setBools[x][y], False)
    posBools[1] = np.append(posBools[x], True)
    Field[0][1] = np.append(Field[x][y], monster)
    pHand = np.delete(pHand, monster)


def flipSummon(player, Field, setBools, posBools, monsters):
    if player == 1:
        x = 0
        y = 1
    elif player == 2:
        x = 1
        y = 0
    for i in range(len(Field[x][y])):
        for monster in monsters:
            if Field[x][y][i] == monster:
                setBools[x][y][i] = False
                posBools[x][i] = True


def setMonster(player, pHand, Field, setBools, posBools, monster, GY):
    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0
    if len(Field[x][y]) > 5:
        min = monster.attack
        minMon = monster
        for mon in Field[x][y]:
            if mon.attack < min:
                min = mon.attack
                minMon = mon
        if monster == minMon:
            return None
        np.delete(Field[0][1], minMon)
        np.append(GY, minMon)
    controlledMonsters = len(Field[x][y])
    setBools[1][0] = np.append(setBools[x][y], True)
    posBools[1] = np.append(posBools[x], False)
    Field[0][1] = np.append(Field[x][y], monster)
    pHand = np.delete(pHand, monster)


# Game Phases
def draw():
    pass


def standby():
    pass


def main1():
    pass


def battle():
    pass


def main2():
    pass


def end():
    pass


# Other Game mechanics
def changePosition(player, Field, posBools, monsters):
    if player == 1:
        x = 0
        y = 1
    elif player == 2:
        x = 0
        y = 1
    monstersControlled = Field[x][y]
    for i in range(len(monstersControlled)):
        for monster in monsters:
            if Field[x][y][i] == monster:
                posBools[x][i] = 1 - posBools[x][i]


def attack(pMonster, opMonster, Field, opGy, pGy):
    pass


def activate():
    pass


def strategy(player, pHand, Field):
    pass


In [57]:
# Card Effects

# examples = 'Monster Reborn', 'Remove Trap', 'Lightning Vortex'
# (cards return None if the activation cost can't be met, or they are useless)

# cards that still need to be defined
# set 1: Sword of Dark Destruction', 'Book of Secret Arts', 'Dark Hole', 'Dian Keto the Cure Master',
# set 2: 'Fissure', 'Trap Hole', 'De-Spell', 'Reinforcements',
# set 3: 'Change of Heart', 'Waboku', 'Soul Exchange', 'Dragon Capture Jar'
# set 4: 'Castle Walls', 'Ultimate Offering', 'Sakuretsu Armor', 'Sogen'
# set 5: 'Dark Energy', 'Invigoration', 'Ookazi', 'Mystical Space Typhoon',


# special someone monster from either GY
def monsterReborn(player, Field, GY, opGY):
    max = 0
    maxMonster = None

    # initialize player field
    if player == 1:
        x = 0
        y = 1
    elif player == 2:
        x = 1
        y = 0

    # if either Graveyard has cards and your field isnt full
    if (len(GY) > 0 or len(opGY) > 0) and len(Field[x][y]) <= 5:
        op = False

        # search for best monster in either graveyard
        for card in GY:
            cardType = re.search("monster", card.type.lower())
            if (cardType != None) and (card.attack > max):
                max = card.attack
                maxMonster = card
        for card in opGY:
            cardType = re.search("monster", card.type.lower())
            if (cardType != None) and (card.attack > max):
                max = card.attack
                maxMonster = card
                op = True

        # if there exists such a monster summon it
        np.append(Field[x][y], maxMonster)
        if maxMonster != None and op == False:
            np.delete(GY, maxMonster)
        elif maxMonster != None and op == True:
            np.delete(opGY, maxMonster)

    # if you can't summon a monster return None
        else:
            return None
    else:
        return None


def removeTrap(player, Field, opGY):
    if player == 1:

        # set (x, y) values to opposite oppenents spell/trap zone
        x = 1
        y = 1

    elif player == 2:

        # set (x, y) values to opposite oppenents spell/trap zone
        x = 0
        y = 0

    # if opponent has a spell or trap on the field
    if len(Field[x][y]) > 0:

        # search for a trap to destroy
        for card in Field[x][y]:
            cardType = re.search("trap", card.type.lower())
            if (cardType != None):
                trap = card
                break

        # destroy trap
        np.append(opGY, trap)
        np.delete(Field[x][y], trap)

    else:

        # return None if no traps can be destroyed
        return None


def LightningVortex(player, Field, hand, GY, opGY):

    # set field (x, y) values to opposite players field
    if player == 1:
        x = 1
        y = 0
    else:
        x = 0
        y = 1

    if len(hand) > 0 and len(Field[x][y]) > 0:
        minAtk = float('inf')
        minCard = hand[0]

        # search for weakest card in hand to discard
        for card in hand:
            cardType = re.search("monster", card.type.lower())
            if cardType != None and card.attack < min:
                minCard = card
                min = card.attack

        # discard 1 card for activation cost
        index = hand.indexOf(minCard)
        np.delete(hand, minCard)
        np.append(GY, minCard)

        # destroy each monster on the opponents field
        for mon in Field[x][y]:
            np.append(GY, mon)
            np.delete(Field[x][y], mon)

    # if activation cost can't be met or opponent has no monsters on their field
    else:
        return None


In [225]:
def SwordOfDarkDestruction(player, Field):
    # Equip only to a DARK monster. It gains 400 ATK and loses 200 DEF.

    # Most Optimal is to equip it to Player. The opponent side doesn't matter since 
    # increase the attack on player will outweight of decreasing defense in opponent side
    
    AttackIncrease = 400
    DefenseDecrease = 200   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    dark_monsters = []
    for monster in Field[x][y]:
        if monster.attribute == "DARK":
            dark_monsters.append(monster)

    if not (dark_monsters):
        print("No DARK monsters on the field to equip the 'Sword of Dark Destruction' to!")
        return None
    else: 
        optimal_monster = max(dark_monsters, key=lambda monster: monster.attack)
        print("Equipping 'Sword of Dark Destruction' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.attack += AttackIncrease
        optimal_monster.defense -= DefenseDecrease
        
     


def BookOfSecretArts(player, Field):
   # A Spellcaster-Type monster equipped with this card increases its ATK and DEF by 300 points

    # Most Optimal is to equip it to Player monster 
    AttackIncrease = 300
    DefenseDecrease = 300   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    spellcaster_monsters = []
    for monster in Field[x][y]:
        if monster.race == "Spellcaster":
            spellcaster_monsters.append(monster)

    if not (spellcaster_monsters):
        print("No SpellCaster monsters on the field to equip the 'Book Of Secret Arts' to!")
        return None
    else: 
        optimal_monster = max(spellcaster_monsters, key=lambda monster: monster.attack)
        print("Equipping 'Book Of Secret Arts' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.attack += AttackIncrease
        optimal_monster.defense -= DefenseDecrease


def DarkHole(player, Field, GY, opGY):
    # Destroy all monsters on the field.
    monsterField1 = Field[0][1]
    monsterField2 = Field[1][0]

    # Return None if nothin on Field
    if (len(monsterField1) == 0 and len(monsterField2) == 0):
        return None

    # Send all monsters to GY
    if (len(monsterField1) > 0):
        for monster in (monsterField1):
            GY.append(monster)
            
    if (len(monsterField2) > 0):
        for monster in (monsterField2):
            opGY.append(monster)
    Field[0][1] = []  
    Field[1][0] = []  
def DianKetotheCureMaster(player, lP1):
    # Increase your Life Points by 1000 points.
    lP1 += 1000
    return lP1
    

In [235]:
def printField(Field):

    for row in Field:
        print('+' + '-'*10 + '+' + '-'*10 + '+')
        for col in row:
            for card in col:
                print(card.name,end=", ")
            print('\n')   
    print('+' + '-'*10 + '+' + '-'*10 + '+')

In [236]:
card = yugioh.get_card(card_name="Dark Magician")
card2 = yugioh.get_card(card_name="Dark End Dragon")
card3 = yugioh.get_card(card_name="Shiranui Squire")
card4 = yugioh.get_card(card_name="Armored Zombie")

Field = np.empty((2, 2), dtype=object)
# fill the array with empty lists
Field.fill([])
Field[0][1] = np.append(Field[0][1], card)
Field[0][1] = np.append(Field[0][1], card2)
Field[1][0] = np.append(Field[1][0], card3)
Field[1][0] = np.append(Field[1][0], card4)
lP1 = 1000
GY = []
OpGy = []

print("GY" , GY)
print("OPGY" , OpGy)
print("LP", lP1)
print("Card Attack",card2.name,"=",card2.attack)
print("Card Attack",card.name,"=",card.attack)
#DarkHole(1, Field, GY, OpGy)
#lP1 = DianKetotheCureMaster(1,lP1)
#SwordOfDarkDestruction(1,Field)
BookOfSecretArts(1,Field)
# letsDuel(p1Deck, p2Deck)
printField(Field)
print("GY" , GY)
print("OPGY" , OpGy)
print("LP", lP1)
print("Card Attack",card2.name,"=",card2.attack)
print("Card Attack",card.name,"=",card.attack)

GY []
OPGY []
LP 1000
Card Attack Dark End Dragon = 2600
Card Attack Dark Magician = 2500
Equipping 'Book Of Secret Arts' to Dark Magician to increase its attack by 300 and decrease its defense by 300
+----------+----------+


Dark Magician, Dark End Dragon, 

+----------+----------+
Shiranui Squire, Armored Zombie, 



+----------+----------+
GY []
OPGY []
LP 1000
Card Attack Dark End Dragon = 2600
Card Attack Dark Magician = 2800
